In [1]:
%cd ../mocbapy

/Users/mcortes/Documents/projects/cmm-bio/caidos/Markopy/mocbapy/mocbapy


In [2]:
import cobra.test
import mocbapy
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
import pandas as pd

# OLD

# NEW

In [131]:
import cobra
from cobra import Metabolite, Reaction, Model
from cobra.util.array import create_stoichiometric_matrix
from benpy import vlpProblem

class Ecosystem:

    def __init__(self, models, prefixes, community_name = "My community model", community_id= "community",
                 pool_bounds = (0,1000), keep_members=True, print_report = True):
        self.models = [m.copy() for m in models]
        self.size = len(models)
        self.name = community_name
        self.prefixes = prefixes
        self.conflicts = None
        self.cmodel = None
        
        # --- step 1 ---
        # Change rxn & met ids and compartments of member models to prefixes[i]:id, prefixes[i]:compartment
        #
        #  ex_mets: prefixes[i]:mex.id -> mex.id, where mex is a exchange metabolite in model i.    
        ex_mets = dict()
        for member_index in range(self.size):
            i_ex_mets = self._rename_member_model(member_index)
            ex_mets.update(i_ex_mets) 
        self.ex_mets = ex_mets
        # --- step 2 ---
        # Store member models original objective functions
        self.objectives = []
        for m in self.models:
            self.objectives.append(m.objective)
        
        # --- step 3 ---
        # Create new cobrapy Model for the community by merging member models
        cmodel = Model(id_or_model = community_id, name=community_name)
        for m in self.models:
            cmodel.merge(m)
            
        # --- step 4 ---
        # Store community model.
        self.cmodel = cmodel       
 
        
        # --- step 5 ---
        # Add pool to community model (including compartment, shared pool mets, pool exchanges
        #                              and update of member exchange reactions)
        # By default exchanges are set with bounds (-1000,1000). Original exchange bounds are stored
        # in exchange_records.
        # Exchange metabolites with different formulas, charges or names among members are stored
        # in conflicts along with these values.
        self.conflicts = self._add_pool(pool_bounds)
        
        # --- step 6 ----
        # Add compartment names. They get lost during merge (step 3)
        ccomps = cmodel.compartments
        for m in self.models:
            for comp in m.compartments:
                ccomps[comp] = m.compartments[comp]
        cmodel.compartments = ccomps
        
        # optional steps:
        
        # print summary report if print_report
        if print_report:
            self.print_build_report(pool_bounds)
        # keep member models are kept unless keep_members is False:
        if not keep_members:
            self.clean_members()    
    
    def print_build_report(self, pool_bounds):
        """ Prints community construction report """
        models = self.models
        cmodel = self.cmodel
        prefixes = self.prefixes
        print("Created community model from %d member models:")               
 
        print("General stats:")
        for i in range(self.size):
            print("model (%d):" % i) 
            print( "\t id = %s, name = %s , prefix= %s" % (models[i].id,models[i].name, prefixes[i]))
            rxns = len(models[i].reactions)
            comps = len(models[i].compartments)
            mex = 0
            for mid in self.ex_mets:
                if mid.startswith(prefixes[i]):
                    mex +=1
            print( "\t\t reactions = %d" % rxns)
            print( "\t\t exchange metabolites = %d" % mex)
            print( "\t\t compartments = %d" % comps)
        print("community model:")
        print( "\t id = %s, name = %s " % (cmodel.id, cmodel.name))
        print( "\t\t reactions = %d" % len(cmodel.reactions))
        print( "\t\t exchange metabolites = %d" % len(self.pool_mets))
        print( "\t\t compartments = %d" % len(cmodel.compartments))        
        print("Exchange metabolite conflicts (formula, charge, name, id) = %d" % len(self.conflicts))   
        print("Community exchange reaction bounds: %s" % str(pool_bounds))
    
    def clean_members(self):
        """ Deletes member individual models"""
        members = self.models
        self.models = None
        del members
        
    def _rename_member_model(self, member_index):
        """ Updates a member model by renaming its metabolite and reaction ids and also 
            its compartment names and ids. 
            The corresponding community member prefixes are used:
                original_rxn_id --> member_prefix:original_rxn_id
                original_met_id --> member_prefix:original_met_id
                original_compartment_id --> member_prefix:original_compartment_id
                original_compartment_name --> member_prefix original_compartment_name

            member_index: index of member model in self.models list. 
                    Draft community model corresponding to a merge of individual models 
                    (with translated ids).
            ex_mets: dictionary 
                     keys: translated exchange metabolite ids (e.g. glyc_e:model1, glyc_e:model2).
                     values: original exchange metabolite ids (e.g. glyc_e, glyc_e)

            Returns:       
            exchange_mets : dict
                                keys: translated ids of exchange metabolites in member model
                                values: original ids of exchange metabolites in member model            
        """        
        
        model = self.models[member_index]
        model_prefix = self.prefixes[member_index]
        comp_dict = dict()
        exchange_mets = dict()
        for rex in model.exchanges:
            for mex in rex.metabolites:
                new_mex_id = "%s:%s" % (model_prefix,mex.id)            
                exchange_mets[new_mex_id]= mex.id

        for m in model.metabolites:
            mid = "%s:%s" % (model_prefix,m.id)
            m.id = mid
            mcomp = "%s:%s" % (model_prefix,m.compartment)
            if mcomp not in comp_dict:
                comp_dict[mcomp]= "%s %s"   % (model_prefix, model.compartments[m.compartment])
            m.compartment = mcomp
        model.repair()    
        model.compartments = comp_dict

        
        for rxn in model.reactions:
            rid = "%s:%s" % (model_prefix,rxn.id)
            rxn.id = rid
        model.repair()


        #print("model (%d) %s updated with prefix %s" % (member_index, model.id, model_prefix))
        #print("Total exchange metabolites in (%d) %s: %d" % (member_index,model.id,len(exchange_mets)))
        return exchange_mets

    def _build_pool_metabolites(self):
        """ Builts community pool metabolites by getting metabolites names, formulas and charges from 
            individual models. Conflicts where more than one value for either metabolite 
            formula, charge or name are found are stored and return.
            A list of Metabolite objects for pool metabolites is generated.

            self.cmodel: cobra.Model 
                    Draft community model corresponding to a merge of individual models 
                    (with translated ids).
            self.ex_mets: dictionary 
                     keys: translated exchange metabolite ids (e.g. glyc_e:model1, glyc_e:model2).
                     values: original exchange metabolite ids (e.g. glyc_e, glyc_e)

            Returns:   

            pool_mets: list
                       Elements of pool_mets are cobra.Metabolite object for each pool metabolite.    
            balance_conflicts : dict
                                keys: pool metabolites ids of pool metabolites with different 
                                formula,charge or name values.
                                values: dict with formula, charge and name lists.            
        """

        cmodel =self.cmodel
        ex_mets = self.ex_mets
        
        pool_mets = []
        balance_conflicts = dict()

        # Pool exchange metabolites original ids in community member models:
        ex_original_ids = set(ex_mets.values())

        # dictionary to store formulas, charges and names for each exchange metabolite:     
        ex_metadata = {mid: {"formulas":set(), "charges":set(),"names":set()} for mid in ex_original_ids} 

        for ex_new, ex_original in ex_mets.items():
            
            m = cmodel.metabolites.get_by_id(ex_new)
                        
            ex_metadata[ex_original]["formulas"].add(m.formula)
            ex_metadata[ex_original]["charges"].add(m.charge)
            ex_metadata[ex_original]["names"].add(m.name) 
            

         

        # Checking if each exchange metabolites has one or more value por formula, charge, or name: 
        for ex_original,ex_data in ex_metadata.items():    
            formulas =  ex_data['formulas']
            charges  =  ex_data['charges']
            names    =  ex_data['names']

            if len(formulas)!=1 or len(charges) != 1 or len(names) != 1:
                balance_conflicts[ex_original] = ex_data 
                print("Conflicts for exchange metabolite %s:" % ex_original)
                print("Formulas:")
                print(formulas)
                print("Charges:")
                print(charges)
                print("Names:")
                print(names)
                print("----------")


            # Setting id, formula,charge and name for new pool metabolite
            eid = "%s:pool" % ex_original
            eformula = "X"  if len(formulas)== 0 else formulas.pop()
            echarge = "0"  if len(charges)== 0 else charges.pop()
            ename = ex_original if len(names)== 0 else names.pop()

            # pool metabolite Metabolite object
            mpool = Metabolite(eid, formula = eformula, charge = echarge, name = ename, compartment = 'pool')
            pool_mets.append(mpool)            

        return pool_mets, balance_conflicts   

    def _build_pool_reactions(self, pool_bounds = (0,1000)):
        """ Builts community exchange reactions and adds them to the community model. 
            Default bounds are set as (-0,1000) for each community exchange reaction."""
        
        
        cmodel = self.cmodel
        pool_mets = self.pool_mets
        
        pool_rxns = list()
        # One exchange reaction for each pool metabolite. 
        for mid in pool_mets:
            m = self.cmodel.metabolites.get_by_id(mid)
            ex_rxn = Reaction(
                id   = "EX_%s" % m.id,
                name = "Pool %s exchange" % m.name,
                subsystem = "Exchange",
                lower_bound = pool_bounds[0],
                upper_bound = pool_bounds[1],
                objective_coefficient = 0.0
            )
            ex_rxn.add_metabolites({m: -1.0})        
            pool_rxns.append(ex_rxn)
        return pool_rxns

    def _add_pool(self,pool_bounds = (0,1000)):
        """ 
        Adds pool to community model (including compartment, shared pool mets, pool exchanges
                                      and update of member exchange reactions)
         By default pool exchanges are set with bounds (-1000,1000)
         Exchange metabolites with different formulas, charges or names among members are stored
         in conflicts along with these values.        
        """
        cmodel =self.cmodel
        ex_mets = self.ex_mets
        
        compartments = cmodel.compartments
        compartments['pool']="Community pool"


        #list of pool metabolites to add and conflict report dict
        pool_mets, conflicts = self._build_pool_metabolites()

        #adding pool metabolites to community model    
        cmodel.add_metabolites(pool_mets) 
        self.pool_mets= [m.id for m in pool_mets]
        #cmodel.repair()

        #list of exchange reactions to add for each pool metabolite
        pool_exchange_reactions = self._build_pool_reactions(pool_bounds)    

        #updating original exchange reactions from member models:
        # from met_e:model1 <--> to met_e:model1 <--> met_e:pool

        original_exchanges = cmodel.exchanges
        exchange_records = dict()
        for ex in original_exchanges:
            exchange_records[ex.id] = ex.bounds
            for mex in ex.metabolites:
                mpool_id = "%s:pool" % ex_mets[mex.id]
                mpool = cmodel.metabolites.get_by_id(mpool_id)
                ex.add_metabolites({mpool:1.0})
            ex.bounds = (-1000,1000)    
        #cmodel.repair()        

        #adding exchange reactions to community model
        cmodel.add_reactions(pool_exchange_reactions)

        #updating compartment names:
        cmodel.compartments = compartments
        
        #storing original exchange reaction bounds:
        self.exchange_records = exchange_records
        
        #storing pool exchange reactions:
        self.pool_reactions = [r.id for r in pool_exchange_reactions]
        

        return conflicts

    def set_experiment(self, exchange_bounds):
        for rid in exchange_bounds:
            pool_exchange_reaction = self.cmodel.reactions.get_by_id(rid)
            pool_exchange_reaction.bounds = exchange_bounds[rid]
            
            

In [9]:
test_models = list()
test_prefixes = list()

n_test = 2
for i in range(n_test):
    model = cobra.test.create_test_model("ecoli")
    mid = 'coli_' + str(i+1)
    test_models.append(model)
    test_prefixes.append(mid)

In [132]:
test_eco = Ecosystem(test_models, test_prefixes)

Created community model from %d member models:
General stats:
model (0):
	 id = iJO1366, name = None , prefix= coli_1
		 reactions = 2583
		 exchange metabolites = 330
		 compartments = 3
model (1):
	 id = iJO1366, name = None , prefix= coli_2
		 reactions = 2583
		 exchange metabolites = 330
		 compartments = 3
community model:
	 id = community, name = My community model 
		 reactions = 5496
		 exchange metabolites = 330
		 compartments = 7
Exchange metabolite conflicts (formula, charge, name, id) = 0
Community exchange reaction bounds: (0, 1000)
